Setting up Mlflow

In [1]:
# Install the following librairies (it is better to create a venv (or conda) virtual environment first and install these librairies in it)
#!pip install mlflow
#!pip install --upgrade jinja2
#!pip install --upgrade Flask
#!pip install setuptools

In [2]:
# starts an MLflow server locally.
# !mlflow server --host 127.0.0.1 --port 8080

## Using the MLflow Client API

- Initiate a new Experiment.

- Start Runs within an Experiment.

- Document parameters, metrics, and tags for your Runs.

- Log artifacts linked to runs, such as models, tables, plots, and more.

In [4]:
from mlflow import MlflowClient
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor

In [5]:
# In order to connect to the tracking server, we’ll need to use the uri that we assigned the server when we started it.

client = MlflowClient(tracking_uri="http://127.0.0.1:8080")

#it allows programmatic interaction with the MLflow tracking server.

We now have a client interface to the tracking server that can both send data to and retrieve data from the tracking server.

In [6]:
all_experiments = client.search_experiments()

print(all_experiments)

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1742589806074, experiment_id='0', last_update_time=1742589806074, lifecycle_stage='active', name='Default', tags={}>]


### Create an experiment

In [7]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "This is the loan default prediction project. "
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "loan-default-prediction",
    #store_dept": "produce",
    #"team": "stores-ml",
    "project_quarter": "Q1-2025",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
produce_loan_default_experiment = client.create_experiment(
    name="Loan_Default_Prediction_Models_3", tags=experiment_tags
)


In [8]:
# Use search_experiments() to search on the project_name tag key

loan_default_experiment_1 = client.search_experiments(
    filter_string="tags.`project_name` = 'loan-default-prediction'"
)

print(vars(loan_default_experiment_1[0]))

{'_experiment_id': '858358373506321674', '_name': 'Loan_Default_Prediction_Models_3', '_artifact_location': 'mlflow-artifacts:/858358373506321674', '_lifecycle_stage': 'active', '_tags': {'mlflow.note.content': 'This is the loan default prediction project. ', 'project_name': 'loan-default-prediction', 'project_quarter': 'Q1-2025'}, '_creation_time': 1742589887844, '_last_update_time': 1742589887844}


### Create a dataset

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv('Loan_Data.csv')


 # Generate features
df_features = df.iloc[:, 1:7]


df.head()

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0


In [3]:
df.describe()

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4.974577e+06,1.461200,4159.677034,8718.916797,70039.901401,4.552800,637.557700,0.185100
std,2.293890e+06,1.743846,1421.399078,6627.164762,20072.214143,1.566862,60.657906,0.388398
min,1.000324e+06,0.000000,46.783973,31.652732,1000.000000,0.000000,408.000000,0.000000
25%,2.977661e+06,0.000000,3154.235371,4199.836020,56539.867903,3.000000,597.000000,0.000000
50%,4.989502e+06,1.000000,4052.377228,6732.407217,70085.826330,5.000000,638.000000,0.000000
75%,6.967210e+06,2.000000,5052.898103,11272.263740,83429.166133,6.000000,679.000000,0.000000
max,8.999789e+06,5.000000,10750.677810,43688.784100,148412.180500,10.000000,850.000000,1.000000


### Logging the first runs with MLflow

In [11]:
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [12]:
# This function call sets the global tracking URI for the current session.
# It’s a convenient way to configure the tracking server URI without creating a separate client instance.

mlflow.set_tracking_uri("http://127.0.0.1:8080")

In [13]:
# Sets the current active experiment to the "Loan_Default_Prediction_Models" experiment and
# returns the Experiment metadata
loan_default_experiment_1 = mlflow.set_experiment("Loan_Default_Prediction_Models_3")

# Define a run name for this iteration of training.
# If this is not set, a unique name will be auto-generated for your run.
run_name = "load_rf_test"

# Define an artifact path that the model will be saved to.
artifact_path = "rf_loan"

In [14]:
# Split the data into features and target and drop irrelevant date field and target field
X = df_features
y = df["default"]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

params = {
    "n_estimators": 100, 
    "max_depth": 6,          
    "min_samples_split": 10,    
    "min_samples_leaf": 4,     
    "bootstrap": True,        
    "oob_score": False,        
    "random_state": 888,        
}

# Train the RandomForestRegressor
rf = RandomForestClassifier(**params)

# Fit the model on the training data
rf.fit(X_train, y_train)

# Predict on the validation set
y_pred = rf.predict(X_val)

# Calculate error metrics
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1_score = f1_score(y_val, y_pred)

# Assemble the metrics we're going to write into a collection
metrics = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1_score}

# Initiate the MLflow run context
with mlflow.start_run(run_name=run_name) as run:
    # Log the parameters used for the model fit
    mlflow.log_params(params)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)

    # Log an instance of the trained model for later use
    mlflow.sklearn.log_model(
        sk_model=rf, input_example=X_val, artifact_path=artifact_path
    )
    

D:\UNIVERSITE\ML_Ops\mlops_project_part_1_mlflow\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
D:\UNIVERSITE\ML_Ops\mlops_project_part_1_mlflow\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing v

🏃 View run load_rf_test at: http://127.0.0.1:8080/#/experiments/858358373506321674/runs/78fa1bcca03d4e2399b6b86f9bcb42cd
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/858358373506321674


### Saving Model

In [16]:
from joblib import dump

dump(rf, "random_forest_model.pkl")


['random_forest_model.pkl']

In [17]:
from joblib import load


rf_loaded = load("random_forest_model.pkl")

# Check type
print(type(rf_loaded))


<class 'sklearn.ensemble._forest.RandomForestClassifier'>
